# Challenge #7: Creating Cutouts from Astronomical Images

---

## Learning Goals

- Use the `astrocut` package to create cutouts from FITS images and TESS full-frame image cubes.
- Create color images from stacks of cutouts.
- Visualize cutouts using `matplotlib` and `lightkurve`.

## Table of Contents

- [Introduction](#Introduction)
- [Imports](#Imports)
- [Cutouts of FITS Images](#Cutouts-of-FITS-Images)
    - [Image Outputs](#Image-Outputs)
- [Cutouts of TESS Full Frame Image Cubes](#Cutouts-of-TESS-Full-Frame-Image-Cubes)
- [Exercise](#Exercise)

## Introduction

Dr. Nefarious is trying to corrupt the archive by overwhelming it with irrelevant data. To stop him, you’ll need to isolate the scientifically valuable regions of massive astronomical datasets—zooming in on stars, galaxies, and transient events with precision.

You’ll learn how to:

- Extract cutouts from FITS images using the `FITSCutout` class.
- Create cutouts from TESS full-frame images using the `TessFootprintCutout` class.

Each cutout you generate brings us one step closer to locking Dr. Nefarious out and securing the archive’s future.

Time is ticking. Let’s dive in.

## Imports

This notebook uses the following packages:

- `astrocut` for creating cutouts from astronomical images.
- `astropy` for handling FITS files and coordinates.
- `astroquery.mast` for identifying TESS sectors.
- `lightkurve` for visualizing TESS data.
- `matplotlib` for visualizing the cutouts.

In [ ]:
import warnings

import lightkurve as lk
import matplotlib.pyplot as plt
from astrocut import FITSCutout, TessFootprintCutout
from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy.wcs import FITSFixedWarning
from astroquery.mast import Tesscut

# Ignore all FITSFixedWarning warnings
warnings.simplefilter('ignore', FITSFixedWarning)

---

## Cutouts of FITS Images

In this section, we will learn how to create cutouts of FITS images using the `astrocut` library. The Flexible Image Transport System (FITS) is a standard format for astronomical data. Astrocut can generate cutouts from FITS files and return the results in memory or as a written file, depending on the user’s preference.

Let's create a cutout of some stack images from the Pan-STARRS survey. These images are of the same region of the sky, but they use three different broadband filters: i, r, and g. We will create a square cutout around Stephan's Quintet, a visual grouping of five galaxies located in the Pegasus constellation.

First, let's plot one of our input images to see what we are working with. We will use the `astropy.io.fits` module to read the FITS file and `matplotlib` to visualize it.

In [ ]:
input_files = ['s3://stpubdata/panstarrs/ps1/public/rings.v3.skycell/2081/049/rings.v3.skycell.2081.049.stk.i.unconv.fits',
               's3://stpubdata/panstarrs/ps1/public/rings.v3.skycell/2081/049/rings.v3.skycell.2081.049.stk.r.unconv.fits',
               's3://stpubdata/panstarrs/ps1/public/rings.v3.skycell/2081/049/rings.v3.skycell.2081.049.stk.g.unconv.fits']

with fits.open(input_files[0], do_not_scale_image_data=True, ignore_blank=True,fsspec_kwargs={'anon': True}) as hdul:
    hdul.info()
    comp_hdu = hdul[1]
    data = comp_hdu.section[...].astype(float)

plt.imshow(data, cmap='gray')
plt.show()

This entire image is 6274 x 6265 pixels! When processing large images, it is often more efficient to work with cutouts that focus on the region of interest. If you look closely, you can see Stephan's Quintet in the lower left corner of the image. Let's create a cutout that zooms in on this region using the `astrocut`.

To make a cutout from a FITS file or files, we will use the `FITSCutout` class, which accepts the following parameters:
- `input_files`: A list of input image files in the expected format. These can be local file paths, URLs, or S3 URIs.
- `coordinates`: The coordinates at the center of the cutout.
- `cutout_size`: The size of the cutout in pixels or angular units.
- `fill_value`: If the cutout footprint falls outside of the image, this value will be used to fill the missing pixels.
- `extension`: The FITS extension to use for the cutout. If not specified, the first extension with data will be used.
- `single_outfile`: If set to `True`, cutouts are written to a single `HDUList` object. If set to `False`, each cutout is written to a separate `HDUList` object.
- `verbose`: If set to `True`, log messages will be printed to the console.

In [ ]:
coordinates = SkyCoord('339.0053026 33.9632280', unit='deg')  # Central coordinate of the cutout
cutout_size = 800  # Size of the cutout in pixels - single scalar or (width, height)
fits_cutout = FITSCutout(input_files=input_files,
                         coordinates=coordinates,
                         cutout_size=cutout_size,
                         single_outfile=True)

The resulting `FITSCutout` object can be used to access cutout data and metadata. The `cutouts_by_file` attribute is a dictionary that stores the individual cutouts as a list of `CutoutInstance` objects by input filename. The `CutoutInstance` object contains the cutout data, shape, world coordinate system (WCS) and other helpful properties.

The `hdu_cutouts_by_file` attribute is a dictionary that stores the cutouts as a list of `ImageHDU` objects by input filename. These objects can be used to access cutout data and metadata in the HDU header.

The `fits_cutouts` attribute is a list of cutouts as `HDUList` objects. The `single_outfile` parameter decides whether all the cutouts will be written to a single HDUList object or to separate objects. When `single_outfile` is set to `True`, a single `HDUList` object will contain all the cutouts.

In [ ]:
cutout = fits_cutout.fits_cutouts[0]
cutout.info()

As you can see, each cutout is saved in its own extension in a single `HDUList` object.

Let's access and plot the cutout data for each filter:

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
for i, (ax, filter) in enumerate(zip(axes, ['i', 'r', 'g'])):
    ax.imshow(cutout[i + 1].data, cmap='gray')
    ax.set_title(f'Filter: {filter}')
    ax.set_xlabel('RA')
    ax.set_ylabel('Dec')

### Image Outputs

The `FITSCutout` class provide methods to normalize the cutout data and write it as an image, either as an `Image` object or a file.

To create cutouts as `Image` objects, use the `get_image_cutouts` method. You can provide the following normalization parameters:
- `stretch`: The stretch function to apply to the image array. Options include “asinh”, “sinh”, “sqrt”, “log”, and “linear”.
- `minmax_percent`: Defines an interval for scaling the image based on percentiles. The format is [lower percentile, upper percentile], where pixel values outside this range are clipped. Only one of `minmax_percent` and `minmax_value` should be specified.
- `minmax_value`: Defines an interval for scaling the image based on values. The format is [min value, max value], where pixel values outside this range are clipped. Only one of `minmax_percent` and `minmax_value` should be specified.
- `invert`: A boolean value that determines whether the image should be inverted.

In [ ]:
images = fits_cutout.get_image_cutouts(stretch='linear', minmax_percent=[0, 100])
images

In [ ]:
# Display the image in the i-band filter
i_img = images[0]
i_img

To produce a colorized RGB image, set the `colorize` parameter to `True`. Color images require three cutouts, which will be treated as the R, G, and B channels, respectively.

In [ ]:
color_img = fits_cutout.get_image_cutouts(stretch='linear', minmax_percent=[0, 100], colorize=True)[0]
color_img

## Cutouts of TESS Full Frame Image Cubes

Astrocut allows you to create cutouts from image cube files, including TESS full-frame images (FFIs).

The `TessFootprintCutout` class generates cutouts from TESS image cube files stored in MAST’s AWS Open Data Bucket. Simply provide the target coordinates and cutout size, and the class will match the cutout’s footprint to the footprints of available cube files on the cloud. A cutout target pixel file will be generated for each matching cube file.

Let's create a cutout of a TESS full-frame image cube centered on the coordinates of a star in the TESS Input Catalog (TIC). First, we'll use the `astroquery.mast.Tesscut` class to identify the sectors that have observed the target star.

In [ ]:
tic_coord = SkyCoord('315.8821398 59.4306840', unit='deg')  # Coordinates for TIC 375422201
Tesscut.get_sectors(coordinates=tic_coord)

To create the cutout, we will use the `TessFootprintCutout` class, which accepts the following parameters:

- `coordinates`: The coordinates at the center of the cutout.
- `cutout_size`: The size of the cutout in pixels or angular units.
- `sequence`: The TESS sector number or a list of sector numbers to restrict the cutouts to specific sectors. If set to `None`, cutouts will be returned for all matching cube files.
- `verbose`: If set to `True`, log messages will be printed to the console.

We will use a cutout size of 10 pixels and only request the cutout from sector 15.

In [ ]:
cutout_size = 10  # Size of the cutout in pixels
sequence = 15  # TESS sector (or list of sectors)
tess_cutout = TessFootprintCutout(coordinates=tic_coord, 
                                  cutout_size=cutout_size, 
                                  sequence=sequence)

The resulting `TessFootprintCutout` object can be used to access the cutout data and metadata. The `cutouts_by_file` attribute is a dictionary that stores the individual cutouts as a list of `CubeCutoutInstance` objects by input filename. The `CubeCutoutInstance` object contains the cutout data, shape, world coordinate system (WCS), and other helpful properties.

The `tpf_cutouts_by_file` attribute is a dictionary that stores the cutouts as a list of `HDUList` objects by input filename. These objects are in the format of target pixel files and can be used to access cutout data and metadata in the HDU header. The `tpf_cutouts` attribute is a list of cutouts as `HDUList` objects in the format of target pixel files.

In [ ]:
tpf_cutout = tess_cutout.tpf_cutouts[0]
tpf_cutout.info()

To better visualize this cutout TPF, we can use the `lightkurve` package, which provides tools for working with TESS data. The `lightkurve` package can read the `HDUList` object and plot the timeseries data as an animation. If you watch closely, you can see the star's brightness change periodically.

In [ ]:
# Animate the cutout TPF with lightkurve
tpf = lk.TessTargetPixelFile(tpf_cutout)
tpf.animate()

## Exercise

To unlock the clue for this challenge, you need to create cutouts from another set of Pan-STARRS stack images. Follow the steps outlined in the sections above to generate the cutouts and visualize them.

We will create a cutout around NGC 1300, a barred spiral galaxy located in the constellation Eridanus. The input image paths, coordinates, and cutout size are provided below. You should save the cutouts in a single `HDUList` object. Use the `FITSCutout` class to generate cutouts for the i, r, and g.

In [ ]:
input_files = ['s3://stpubdata/panstarrs/ps1/public/rings.v3.skycell/0892/015/rings.v3.skycell.0892.015.stk.i.unconv.fits',
               's3://stpubdata/panstarrs/ps1/public/rings.v3.skycell/0892/015/rings.v3.skycell.0892.015.stk.r.unconv.fits',
               's3://stpubdata/panstarrs/ps1/public/rings.v3.skycell/0892/015/rings.v3.skycell.0892.015.stk.g.unconv.fits']

coordinates = SkyCoord('49.9210259 -19.4111632', unit='deg')  # Coordinates for NGC 1300
cutout_size = (900, 600)  # Size of the cutout in pixels (width, height). Feel free to change this!

# Create the cutouts here with `FITSCutout`!
fits_cutout = ...

Next, access the `HDUList` object associated with the cutout, and print it's info.

In [ ]:
# Access the `HDUList` object and print its info


Choose one of the cutouts and plot it using `matplotlib`.

In [ ]:
# Choose one of the cutouts to diplay with `matplotlib`


Generate a RGB color image from the cutouts using the `get_image_cutouts` method. Play around with the different normalization parameters to see how they affect the final image.

In [ ]:
# Generate a colorized image from the cutouts and display it


Hmmm, the shape of this galaxy reminds me of a certain letter... I think we've just found our clue! 

Congratulations cutout master! You're one step closer to stopping Dr. Nefarious.

## Additional Resources

- [Astrocut Documentation](https://astrocut.readthedocs.io/en/latest/index.html)

## About this Notebook

**Author:** Sam Bianco <br>
**Keywords:** AAS 246, Astrocut, Cutouts, PS1, TESS <br>
**Last Updated:** June 2025 <br>
***
[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 